In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import warnings

import gseapy as gp
from gseapy.plot import barplot, dotplot
import gffpandas.gffpandas as gffpd

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

path_to_data='/Users/aleksandradenisova/Desktop/coursework/data/'

In [49]:
order = pd.read_csv(f'{path_to_data}stage4/sorted_inno_sp.csv')
order = order.rename(columns = {'VGL ID':'sp2'})
order.head()

,TotInno_resResEff2022,sp2
0,1.974601,bHirRus1
1,1.950304,bAquChr1
2,1.565219,bCorMon1
3,1.445271,bEriRub2
4,1.357739,bSteHir1


In [50]:
table_name = 'maf_counts_cated.csv'
cons_in_table = ['cons','inno','non_inno']

all_dfs = []
for i in range(3):
    df = pd.read_csv(f'{path_to_data}final_data/{table_name}',
                     names=['start', 'cons', 'inno', 'non_inno', 'Ref_inno', 
                              'Alt_noninno','n_inno', 'n_noninno', 'n', 'inno_N', 
                              'inno_-', 'noninno_N', 'noninno_-', 'zebra', 'chrom'])
    df['start'] = df['start']+1 
    df['type'] = cons_in_table[i]
    
    print(f'number of positions:  {df.shape[0]}')
    
    df3 = df[(((df['n_inno']==4)&(df['n_noninno']==4)) |
              ((df['n_inno']==3)&(df['n_noninno']==4))|
              ((df['n_noninno']==3)&(df['n_inno']==4))) #&((df['noninno_N']==1)|(df['noninno_-']==1))))
              &(df[cons_in_table[i]]==True)]

    print(f'number of positions:  {df3.shape[0]}')
    
    all_dfs.append(df3)
    
df = pd.concat(all_dfs)

number of positions:  36577
number of positions:  3263
number of positions:  36577
number of positions:  3809
number of positions:  36577
number of positions:  3040


In [51]:
df_chrom = pd.read_csv('~/Desktop/coursework/data/stage2/chroms_zebra.tsv', sep = '\t')[:-1]

df_chrom['Molecule name'] = df_chrom['Molecule name'].apply(lambda x: x.split()[1])
df1 = df_chrom[['RefSeq sequence', 'Molecule name']].rename(columns= {'RefSeq sequence':'chrom'})
df1.head()

,chrom,Molecule name
0,NC_044211.2,1
1,NC_044212.2,1A
2,NC_044213.2,2
3,NC_044214.2,3
4,NC_044215.2,4


In [52]:
types = ['Ref_inno', 'Alt_noninno']

for num in range(2):    
    df_p = df

    df2 = pd.merge(df1,df_p, on='chrom').rename(columns = {
                                                        'chrom':'#CHROM', 'start':'POS', 
                                                        'zebra':'REF', types[num]:'ALT'})
    df2['ID'] = '.'
    df2['FILTER'] = '.'
    df2['INFO'] = '.'
    df2['POS'] = df2['POS']

    df3 = df2[df2['REF']!=df2['ALT']][['#CHROM', 'POS','ID',  'REF','ALT', 'FILTER', 'INFO']]

    print(df3.head())
    df3.to_csv(f'{path_to_data}/final_data/positions_for_8_{types[num]}.vcf', 
            index = False,
            sep = '\t')

        #CHROM      POS ID REF ALT FILTER INFO
0  NC_044211.2   407365  .   G   A      .    .
1  NC_044211.2  1153711  .   T   C      .    .
2  NC_044211.2  1305679  .   T   C      .    .
3  NC_044211.2  1481305  .   T   C      .    .
4  NC_044211.2  1837254  .   G   A      .    .
          #CHROM     POS ID REF    ALT FILTER INFO
328  NC_044211.2  139949  .   C    A,C      .    .
329  NC_044211.2  148892  .   A    A,T      .    .
330  NC_044211.2  329839  .   C    G,C      .    .
331  NC_044211.2  854921  .   G    G,C      .    .
332  NC_044211.2  904982  .   G  A,G,C      .    .


### Launched from /Users/aleksandradenisova/Desktop/coursework/code/scripts/scripts/SNP_analysis

sh bash_snpEff.sh final_data positions_for_8_Ref_inno.vcf 2> ~/Desktop/coursework/data/final_data/ref_inno_zibra.log

sh bash_snpEff.sh final_data positions_for_8_Alt_noninno.vcf 2> ~/Desktop/coursework/data/final_data/alt_noninno_zibra.log

### Cобираем мутации из полученных файлов

In [43]:
df_inno_and_non = []

types = ['Ref_inno', 'Alt_noninno']

for num in range(2):
    df_ann = pd.read_csv(f'{path_to_data}/final_data/positions_for_8_{types[num]}.vcf.ann', 
                         skiprows=6, sep = '\t', names = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'FILTER', 'smth','INFO'])

#    df_ann = df_ann[df_ann['REF']!=df_ann['ALT']].reset_index(drop=True)
    ans = []
    for idx in df_ann.index:
        new_info =  df_ann['INFO'].iloc[idx].split('|,')

        for inf in new_info:
            tt = df_ann.iloc[idx]
            tt['info'] = inf
            ans.append(tt)

    df_new = pd.DataFrame(ans).drop(columns = 'INFO').reset_index(drop=True)
    df_new['type'] = df_new['info'].apply(lambda x: x.split('|')[1])
    df_new['gene_name'] = df_new['info'].apply(lambda x: x.split('|')[3])
    df_new['effect'] = df_new['info'].apply(lambda x: x.split('|')[2])
    df_new['type'] = types[num]
    df_inno_and_non.append(df_new)

df_non_and_inno = pd.concat(df_inno_and_non)

In [44]:
inno = df_non_and_inno[df_non_and_inno['type']=='Ref_inno']
non_inno = df_non_and_inno[df_non_and_inno['type']=='Alt_noninno']

In [45]:
len(set(non_inno['POS'].unique()) - set(inno['POS'].unique()))

0

In [46]:
non_inno.head()

,#CHROM,POS,ID,REF,ALT,FILTER,smth,info,type,gene_name,effect
0,NC_044211.2,139949,.,C,"A,C",.,.,ANN=A|intron_variant|MODIFIER|CMSS1|CMSS1|tran...,Alt_noninno,CMSS1,MODIFIER
1,NC_044211.2,139949,.,C,"A,C",.,.,A|intron_variant|MODIFIER|FILIP1L|FILIP1L|tran...,Alt_noninno,FILIP1L,MODIFIER
2,NC_044211.2,139949,.,C,"A,C",.,.,A|intron_variant|MODIFIER|FILIP1L|FILIP1L|tran...,Alt_noninno,FILIP1L,MODIFIER
3,NC_044211.2,148892,.,A,"A,T",.,.,ANN=T|intron_variant|MODIFIER|CMSS1|CMSS1|tran...,Alt_noninno,CMSS1,MODIFIER
4,NC_044211.2,148892,.,A,"A,T",.,.,T|intron_variant|MODIFIER|FILIP1L|FILIP1L|tran...,Alt_noninno,FILIP1L,MODIFIER


In [47]:
inno.head()

,#CHROM,POS,ID,REF,ALT,FILTER,smth,info,type,gene_name,effect
0,NC_044211.2,407365,.,G,A,.,.,ANN=A|intron_variant|MODIFIER|TMEM45A|TMEM45A|...,Ref_inno,TMEM45A,MODIFIER
1,NC_044211.2,407365,.,G,A,.,.,A|intron_variant|MODIFIER|TMEM45A|TMEM45A|tran...,Ref_inno,TMEM45A,MODIFIER
2,NC_044211.2,1153711,.,T,C,.,.,ANN=C|intergenic_region|MODIFIER|TRNAI-AAU-LOC...,Ref_inno,TRNAI-AAU-LOC121470433,MODIFIER
3,NC_044211.2,1305679,.,T,C,.,.,ANN=C|intergenic_region|MODIFIER|LOC121470433-...,Ref_inno,LOC121470433-ALCAM,MODIFIER
4,NC_044211.2,1481305,.,T,C,.,.,ANN=C|intergenic_region|MODIFIER|LOC121470433-...,Ref_inno,LOC121470433-ALCAM,MODIFIER
